### Практическое задание
В этом практическом задании мы попробуем решить задачу прогнозирования временного ряда, а именно выручки нашего сервиса "через 30 дней".

Обычно аналитику того или иного сервиса интересно делать предсказание тренда, то есть средних или кумулятивных значений метрики. Тем не менее иногда нужен и точечный прогноз. Например, для оценки потенциальной нагруженности сервиса в те или иные дни. Точечный ответ в данном случае может помочь принять своевременные решения о проведении плановых работ.

Задача, которую мы сейчас планируем решать, имеет некоторую особенность: из данных у нас будет только история значений выручки в предыдущие дни. Иными словами, мы хотим прогнозировать временной ряд.

В этом задании мы потренируемся выгружать данные, создавать признаки, считать метрики, залезать внутрь обученной модели и искать альтернативные пути. Заложим основу для того, что будем делать дальше, и даже обнаружим некоторые вещи, которых нам сейчас не хватает. Поехали!

### Задача 1. Выгрузка данных
Вам предстоит работать с данными классифайда – таблицей `user_transactions`, которая доступна в Clickhouse (в базе hardda).

##### Запрос для получения данных
Соберём датасет с суммарной выручкой за каждый день. Для этого напишем SQL-запрос к указанной таблице:

- Возьмите только транзакции трёх типов: `'basic sale'`, `'fast sale'`, `'quick sale'`. По датам фильтровать не нужно: берём все доступные даты. Фильтр по столбцу `sign` тоже не нужен: нас будет интересовать чистая выручка за вычетом возвратов и т.п.
- Сгруппируйте по `payment_date`. Чуть позже, в другой части задания, мы поработаем с данными отдельно по каждому типу транзакции. И будем группировать как по дате, так и по типу. Но тут пока будем использовать данные с суммарной выручкой, без разбиения на отдельные типы.
- Для каждой даты посчитайте `-sum(amount)`, получившийся столбец назовите `volume`. Обратите внимание, что `amount` в таблице лежит с отрицательным знаком, т.к. деньги списываются со счета. Мы инвертируем знак, чтобы привести к более интуитивно понятному виду.
- Отсортируйте результат по `payment_date`, чтобы даты шли по порядку.

Запрос можно выполнять непосредственно с помощью `client.execute`, но может пригодиться вспомогательная функция, возвращающая результат сразу в виде датафрейма:

После запроса примените `pd.to_datetime` к столбцу с датами.

На данном этапе у вас должен получиться датафрейм с 2 столбцами. А сколько в нём получилось строк?

In [20]:
import pandas as pd
import numpy as np
from clickhouse_driver import Client
from config import user_id, password
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# работаем с двумя БД, поэтому создаем два подключения.
readDB = 'hardda'
readWriteDB = 'hardda_student_data'


# Создаем соединение с ClickHouse
client = Client(
    host='clickhouse.lab.karpov.courses',
    port=9000,
    user=user_id,
    password=password,
    database=readDB
)

def get_data(query):
    """
    Вытягивает данные из clickhouse в виде Dataframe
    
    query - запрос
    """
    result, columns = client.execute(query, with_column_types=True)
    return pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

In [2]:
query = """
    SELECT payment_date, 
      -SUM(amount) AS volume
    FROM user_transactions
    WHERE type IN ('basic sale', 'fast sale', 'quick sale')
    GROUP BY payment_date
"""

df = get_data(query)
df.shape[0]

511

### Задача 2. Исправление проблемы в данных
Посмотрите, всё ли в порядке с полученным датафреймом (подсказка: если вызвать .head(), вроде бы всё ок, но проблема видна, если вызвать .head(10)).

В датафрейме отсутствуют некоторые даты (например, 2021-02-08).

В реальной жизни мы бы обязательно постарались разобраться в природе этого. Возможно, у нас неполные или некорректные данные, и можно получить более полные. Возможно, транзакции в эти дни были, но по какой-то причине данных по ним нет. Тогда можно было бы попробовать заполнять эти пропуски (например, средним или предыдущим значением).

В нашей задаче будем считать, что транзакций в эти дни действительно не было. В этом случае нужно сделать так, чтобы отсутствующие даты появились в датафрейме со значением 0.

Добавьте пропущенные даты со значением volume равным 0. После этого снова вызовите .head(10) и убедитесь, что строка с датой 2021-02-08 теперь есть.

Сколько строк в датафрейме после добавления пропущенных дат?

In [3]:
df['payment_date'] = pd.to_datetime(df['payment_date'], format='%Y-%m-%d')

In [4]:
df.head(10)

,payment_date,volume
0,2021-02-02,3396867
1,2021-02-03,3504675
2,2021-02-04,3321098
3,2021-02-05,3317318
4,2021-02-06,2763316
5,2021-02-07,2996701
6,2021-02-09,2956741
7,2021-02-10,3053434
8,2021-02-11,2854137
9,2021-02-12,3030957


In [5]:
missing_dates = (
    pd.date_range(start=df['payment_date'].min(), end=df['payment_date'].max())
    .difference(df['payment_date'])
)

missing_dates

DatetimeIndex(['2021-02-08', '2021-02-13', '2021-02-19', '2021-02-20',
               '2021-02-24', '2021-02-26', '2021-02-28', '2021-03-11',
               '2021-03-20', '2021-03-21',
               ...
               '2023-01-03', '2023-01-05', '2023-01-06', '2023-01-09',
               '2023-01-13', '2023-01-19', '2023-01-20', '2023-01-22',
               '2023-01-24', '2023-01-25'],
              dtype='datetime64[ns]', length=218, freq=None)

In [6]:
all_dates = pd.date_range(start=df['payment_date'].min(), end=df['payment_date'].max())
all_dates

DatetimeIndex(['2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05',
               '2021-02-06', '2021-02-07', '2021-02-08', '2021-02-09',
               '2021-02-10', '2021-02-11',
               ...
               '2023-01-22', '2023-01-23', '2023-01-24', '2023-01-25',
               '2023-01-26', '2023-01-27', '2023-01-28', '2023-01-29',
               '2023-01-30', '2023-01-31'],
              dtype='datetime64[ns]', length=729, freq='D')

In [7]:
df2 = all_dates.to_frame(name='payment_date').merge(df, on='payment_date', how='left').fillna(0)
df2['volume'] = df2['volume'].astype(int)
df2.head(10)

,payment_date,volume
0,2021-02-02,3396867
1,2021-02-03,3504675
2,2021-02-04,3321098
3,2021-02-05,3317318
4,2021-02-06,2763316
5,2021-02-07,2996701
6,2021-02-08,0
7,2021-02-09,2956741
8,2021-02-10,3053434
9,2021-02-11,2854137


In [8]:
df2.shape[0]

729

### Задача 3. Подготовка признаков

Инструменты ML очень универсальны и могут применяться в том числе и в прогнозировании TS (Time Series). Многие из вас ранее могли уже работать с TS и использовать классические эконометрические подходы. Вы, наверное, не удивитесь, когда узнаете, что под капотом этих подходов зачастую используется линейная регрессия, поэтому не стоит игнорировать этот базовый подход, когда работаете с данными с временной структурой.

Есть целое семейство стратегий "как свести задачу прогнозирования TS к классической регрессионной ML задаче". Подход, который мы предлагаем реализовать вам — лишь один из многих.

##### В чём идея

Наша задача — предсказать выручку в конкретный день через 30 дней. Например, представим, что сейчас 1 августа, а мы хотим предсказать выручку за 31 августа. Что мы можем для этого использовать:

- Мы знаем, какой будет день недели 31 августа. Из этого можно вытащить какие-то признаки.
- Можно использовать выручку за конкретный день в прошлом. Т.к. мы предсказываем "через 30 дней", нельзя брать данные за дни ближе, чем 30 дней. В нашем примере мы уже знаем выручку за 1 августа, за любой день в июле, июне, мае. Но мы не можем брать выручку за 29 августа, 15 августа или 2 августа. Они ещё не наступили. Таким образом, если мы собираем фичи для таргета 31 августа, мы можем брать дни за 30 дней и более до него: 1 августа и раньше.
- Конечно, из прошлого мы можем брать не только выручку за конкретные дни. Можно посчитать агрегированные или скользящие значения.

##### Признаки
Итак, для каждой даты `payment_date` у нас есть таргет, `volume` (истинное значение выручки). Чтобы предсказать это значение, соберём признаки (используя идеи, которые обсудили выше):

На основе дня недели посчитаем один признак — является ли этот день выходным (1, если суббота или воскресенье; 0, если нет).
Возьмём три значения выручки за какой-то день в прошлом: 30 дней назад, 61 день назад и 91 день назад (выбор конкретных значений тут довольно произвольный). Это ещё три признака.
Помимо "точечных" значений за конкретные дни в прошлом, возьмём экспоненциальные скользящие средние. Начиная с дня 30 дней назад и дальше в прошлое (на 7 дней, на 30 дней и на 91 день).
В некоторых случаях может не хватить глубины данных, чтобы посчитать все признаки. Это будет выглядеть как признаки со значением `NaN`. Строки, содержащие такие значения, мы просто удалим (могут быть и другие подходы, но мы используем самый простой).

Итого, для каждой даты у нас помимо таргета появится 7 признаков, которые можно использовать для предсказания этого значения.

Дальше решаем как обычную задачу обучения с учителем. Такой сетап позволит нам прогнозировать на 30 дней вперед из любой доступной на текущий момент даты.

##### Функция для подготовки признаков

Обязательно попробуйте самостоятельно посчитать хотя бы часть признаков, описанных выше.

Но для итогового решения можете взять готовую функцию `generate_features_for_series`, которая делает всё описанное.

Убедитесь, что понимаете, как она работает. Не забудьте перед её применением добавить отсутствующие даты (см. предыдущий степ).

```python
def generate_features_for_series(
    data: pd.DataFrame,
    date_colname='payment_date',
    value_colname='volume'
):


    data_ = data.copy()
    data_.sort_values(date_colname, inplace=True)
   
    ### 1)
    data_['weekday'] = data_[date_colname].dt.weekday
    data_.loc[data_['weekday'].isin([5, 6]), 'is_holiday'] = 1
    data_['is_holiday'] = data_['is_holiday'].fillna(0)
    data_.drop('weekday', axis=1, inplace=True)


    ### 2, 3, 4)
    data_['lag_30d'] = data_[value_colname].shift(30)
    data_['lag_61d'] = data_[value_colname].shift(61)
    data_['lag_91d'] = data_[value_colname].shift(91)


    ### 5, 6, 7)
    data_['rolling_7d'] = data_['lag_30d'].ewm(span=7).mean()
    data_['rolling_30d'] = data_['lag_30d'].ewm(span=30).mean()
    data_['rolling_91d'] = data_['lag_30d'].ewm(span=91).mean()


   
    ### Уберем Nan'ы
    data_.dropna(inplace=True)
   
    return data_
```

Обратите внимание, что при проверке, что день недели является выходным, используем [5, 6], а не [6, 7], т.к. нумерация дней недели начинается с нуля (см. pandas.Series.dt.weekday).

 Сколько строк получилось после применения функции генерации признаков?

In [9]:
def generate_features_for_series(
        data: pd.DataFrame,
        date_colname='payment_date',
        value_colname='volume'
    ):

    data_ = data.copy()
    data_.sort_values(date_colname, inplace=True)
   
    ### 1)
    data_['weekday'] = data_[date_colname].dt.weekday
    data_.loc[data_['weekday'].isin([5, 6]), 'is_holiday'] = 1
    data_['is_holiday'] = data_['is_holiday'].fillna(0)
    data_.drop('weekday', axis=1, inplace=True)

    ### 2, 3, 4)
    data_['lag_30d'] = data_[value_colname].shift(30)
    data_['lag_61d'] = data_[value_colname].shift(61)
    data_['lag_91d'] = data_[value_colname].shift(91)

    ### 5, 6, 7)
    data_['rolling_7d'] = data_['lag_30d'].ewm(span=7).mean()
    data_['rolling_30d'] = data_['lag_30d'].ewm(span=30).mean()
    data_['rolling_91d'] = data_['lag_30d'].ewm(span=91).mean()

   
    ### Уберем Nan'ы
    data_.dropna(inplace=True)
   
    return data_

In [10]:
data = generate_features_for_series(df2)

In [11]:
data

,payment_date,volume,is_holiday,lag_30d,lag_61d,lag_91d,rolling_7d,rolling_30d,rolling_91d
91,2021-05-04,4585501,0.0,0.0,3298677.0,3396867.0,1.052007e+06,1.837514e+06,2.137795e+06
92,2021-05-05,4842622,0.0,3707972.0,2632327.0,3504675.0,1.715998e+06,1.960023e+06,2.183332e+06
93,2021-05-06,4289897,0.0,3929579.0,2489894.0,3321098.0,2.269393e+06,2.088896e+06,2.233610e+06
94,2021-05-07,0,0.0,3719324.0,2604353.0,3317318.0,2.631876e+06,2.195482e+06,2.276087e+06
95,2021-05-08,0,1.0,0.0,2141489.0,2763316.0,1.973907e+06,2.052080e+06,2.211456e+06
...,...,...,...,...,...,...,...,...,...
724,2023-01-27,7606314,0.0,5576730.0,5560131.0,6605566.0,4.042208e+06,4.284195e+06,4.362926e+06
725,2023-01-28,6243591,1.0,4879158.0,6110948.0,5888153.0,4.251446e+06,4.322580e+06,4.374148e+06
726,2023-01-29,6866600,1.0,3997769.0,6505339.0,6081790.0,4.188027e+06,4.301624e+06,4.365966e+06
727,2023-01-30,8337223,0.0,2698006.0,0.0,0.0,3.815521e+06,4.198165e+06,4.329706e+06


### Задача 4. Обучение модели (1/2)
Наконец добрались до самого интересного. Вам предстоит обучить модель линейной регрессии и замерить качество работы вашей модели!

Вообще говоря, для корректного замера качества нужно было бы разделить выборку на несколько частей. Обучать модель на одних данных, а проверять качество на других данных, которые модель ещё не видела. Но об этом мы поговорим на следующем уроке. А пока просто выполним вычисления на всем датафрейме и найдем MSE и MAE.

- Разделите данные на X и y.
- Создайте экземпляр LinearRegression, обучите его.
- Получите предсказания (пока для тех же X и y). Посчитайте метрики качества.

Округлите значение MSE до миллионов и введите полученное значение ("в миллионах") в поле ниже. Например, если у вас получилось 123 876 000, введите 124.

In [13]:
model = LinearRegression()

In [16]:
X_objects = data.drop(['payment_date', 'volume'], axis=1)
Y_targets = data['volume']

print("Матрица объектов и фичей: \n")
print(X_objects)
print("\n И вектор таргетной переменной: \n")
print(Y_targets)

Матрица объектов и фичей: 

     is_holiday    lag_30d    lag_61d    lag_91d    rolling_7d   rolling_30d  \
91          0.0        0.0  3298677.0  3396867.0  1.052007e+06  1.837514e+06   
92          0.0  3707972.0  2632327.0  3504675.0  1.715998e+06  1.960023e+06   
93          0.0  3929579.0  2489894.0  3321098.0  2.269393e+06  2.088896e+06   
94          0.0  3719324.0  2604353.0  3317318.0  2.631876e+06  2.195482e+06   
95          1.0        0.0  2141489.0  2763316.0  1.973907e+06  2.052080e+06   
..          ...        ...        ...        ...           ...           ...   
724         0.0  5576730.0  5560131.0  6605566.0  4.042208e+06  4.284195e+06   
725         1.0  4879158.0  6110948.0  5888153.0  4.251446e+06  4.322580e+06   
726         1.0  3997769.0  6505339.0  6081790.0  4.188027e+06  4.301624e+06   
727         0.0  2698006.0        0.0        0.0  3.815521e+06  4.198165e+06   
728         0.0        0.0        0.0  7244990.0  2.861641e+06  3.927316e+06   

      rolli

In [17]:
model.fit(X_objects, Y_targets)

LinearRegression()

In [19]:
preds = model.predict(X_objects)

In [21]:
MSE = mean_squared_error(Y_targets, preds)

In [24]:
round(MSE / 1000000)

7659275

### Задача 4. Обучение модели (2/2)
Округлите значение метрики MAE до миллионов и введите полученное значение в поле ниже. Например, если у вас получилось 123 876 000, введите 124.

In [25]:
round(mean_absolute_error(Y_targets, preds) / 1000000)

2

### Задача 5. Асимметричные метрики качества (1/2)
В машинном обучении бывает полезно считать асимметричные метрики. Особенно в случаях, когда мы хотим дать лишь нижнюю или верхнюю оценку при прогнозировании. Прекрасный пример такой метрики — применение квадратичной ошибки при перепрогнозе, а абсолютной при недопрогнозе на каждом объекте (или наоборот). Далее — усреднение посчитанных по каждому объекту loss'ов. Формулу можно было бы записать следующим образом:
$$Asymetric=[Prediction>Target]*(Prediction-Target)^2+[Prediction<Target]*|Prediction-Target|$$

Здесь `[Prediction>Target]` равно 1, если условие выполняется, и 0 в противном случае. В коде можно либо использовать такой же подход, либо просто посчитать нужное выражение только для тех строк, где соответствующее условие выполняется, а потом сложить обе части.

Метрика, упомянутая выше, сильнее штрафует за перепрогноз, чем за недопрогноз, т.к. в случае перепрогноза (предсказание больше, чем таргет) ошибка возводится в квадрат.

Такая метрика может быть полезна, например, при закупке скоропортящихся продуктов. Если мы закупим намного меньше, чем нужно, это неприятно, но решаемо (можно дозаказать, либо в крайнем случае клиент купит альтернативный продукт). Если же мы закупим намного больше, чем нужно, то еда просто испортится.

Посчитайте эту метрику. Округлите значение до миллионов и введите полученное значение в поле ниже.

In [37]:
def assymetric_error(
        targ: np.array,
        pred: np.array,
        punish_mode='overestimation'
    ):
    under_est = (pred - targ)[pred < targ]
    over_est = (pred - targ)[pred > targ]
    
    if punish_mode == 'overestimation':
        return (sum(abs(under_est)) + sum(over_est ** 2)) / pred.shape[0]
    elif punish_mode == 'underestimation':
        return (sum(abs(over_est)) + sum(under_est ** 2)) / pred.shape[0]
    else:
        raise ValueError('Unknown mode')

In [38]:
round(assymetric_error(Y_targets, preds) / 1_000_000)

4838182

### Задача 5. Асимметричные метрики качества (2/2)
Если мы захотим наоборот сильнее штрафовать модель за недопрогноз, чем за перепрогноз, то знаки больше-меньше в формуле изменятся на противоположные:
$$Asymetric=[Prediction<Target]*(Prediction-Target)^2+[Prediction>Target]*|Prediction-Target|$$

Продолжая пример с закупками, предположим теперь, что мы закупаем не скоропортящуюся еду, а жизненно необходимые препараты для больницы. Закупить намного больше необходимого тоже нехорошо, т.к. будет лежать без дела. Но закупить намного меньше, чем нужно, — может стоить жизней. Модель нужно сильнее штрафовать за недопрогноз, чем за перепрогноз.

Подумайте, в каких ещё задачах есть смысл сильнее штрафовать модель за недопрогноз или наоборот за перепрогноз.

Посчитайте и эту асимметричную метрику. Округлите значение до миллионов и введите полученное значение в поле ниже.

In [39]:
round(assymetric_error(Y_targets, preds, punish_mode='underestimation') / 1_000_000)

2821095

### Задача 6. Интерпретация коэффициентов обученной модели (1/3)
Теперь перейдем к интерпретации коэффициентов модели.

У каких топ-3 фичей наибольший негативный вклад в формирование прогноза? Иными словами, выберите те фичи, увеличение которых на единицу (при прочих равных) приведет к наибольшему снижению прогноза.

In [43]:
pd.DataFrame({'feature': X_objects.columns, 'coef': model.coef_}).sort_values('coef').reset_index(drop=True)

,feature,coef
0,is_holiday,-255284.496169
1,rolling_30d,-1.581143
2,lag_91d,-0.073367
3,lag_61d,-0.054020
4,lag_30d,-0.025142
5,rolling_7d,0.320666
6,rolling_91d,2.042672


### Задача 6. Интерпретация коэффициентов обученной модели (2/3)
У нас есть обученная модель. Представим, мы используем её для предсказания выручки в такой день, у которого все признаки равны 1. Чему будет равен прогноз модели в этом случае?

Это вопрос на понимание того, как работают линейные модели. Постарайтесь ответить на него, не вызывая predict. Только на основе коэффициентов обученной модели.

Получив ответ, перепроверьте себя, вызвав `.predict([[1,1,1,1,1,1,1]])`. Убедитесь, что понимаете, почему ответы получаются одинаковыми.

Введите ответ в поле ниже с точностью до трех знаков в десятичной части. Используйте точку для разделения целой и дробной части.

In [51]:
round(model.coef_.sum() + model.intercept_, 3)

1486271.702

### Задача 6. Интерпретация коэффициентов обученной модели (3/3)
В продолжение предыдущего вопроса. Каким прогноз окажется, если выбранный день не был выходным? То есть теперь один из признаков равен 0, т.к. день не выходной.

Так же, как на предыдущем степе, попробуйте ответить, используя только коэффициенты обученной модели и не вызывая `predict`. А потом перепроверьте себя, используя `predict`. И убедитесь, что  понимаете, почему ответы совпадают.

Введите ответ в поле ниже с точностью до трех знаков в десятичной части. Используйте точку для разделения целой и дробной части.

In [54]:
round(model.coef_.sum() - model.coef_[0] + model.intercept_, 3)

1741556.199

### Задача 7. Отдельные модели по типам транзакций (1/2)
Мы везде использовали данные с суммарной выручкой за день. Но в данных у нас были транзакции 3 типов: 'basic sale', 'fast sale', 'quick sale'. Что если эти три типа "ведут себя по-разному", и мы теряем важную информацию, просто складывая их все вместе?

### В чём идея
Попробуем другой подход. Сейчас наш временной ряд (time series, TS) по сути является суммой трёх рядов по типам транзакций:

$$TS=TS_1+TS_2+TS_3$$
 

Например, для даты `2021-02-02` суммарная выручка равна 3396867, но она складывается из 830390 для `basic sale`, 1352338 для `fast sale` и 1214139 для `quick sale`. Так же для остальных дат.

---

Вместо того, чтобы работать с общей суммой, можно попробовать предсказать выручку отдельно по каждому типу, а потом сложить и получить итоговый результат.

Например, если для какой-то даты модель для `basic sale` выдаст предсказание 2 миллиона, модель для `fast sale` предскажет 3 миллиона, а модель для `quick sale` 1 миллион, то можем считать, что итоговым предсказанием для этой даты будет 2 + 3 + 1 = 6 миллионов.

Другими словами, если наш временной ряд TS является суммой рядов $TS_1$, $TS_2$, $TS_3$ и для каждого из них есть своя модель, то общее предсказание можно посчитать как сумму предсказаний этих моделей:
$$a(TS) = a_1(TS_1)+a_2(TS_2)+a_3(TS_3)$$

### Реализация
Заново соберите данные, на этот раз с группировкой не только по дате, но и по типу.

Дальше для каждого типа транзакции нужно повторить все те же шаги, которые были до этого:

1. Возьмите датафрейм с данными по конкретному типу.
2. Добавьте недостающие даты со значением 0. Обратите внимание, что после этого этапа могут появиться пропуски в столбце type, что может привести к некорректной работе функции `generate_features_for_series`. Либо удалите этот столбец совсем (мы его уже использовали для фильтрации данных по типу, и он больше не нужен), либо заполните пропуски в нём.
3. Вызовите функцию `generate_features_for_series` для генерации признаков.
4. Обучите модель.
5. Используйте модель для получения предсказаний (тут снова напоминание, что, вообще говоря, неправильно проверять качество модели на той же выборке, на которой мы обучались, но в этом уроке пока делаем так).
Можете попробовать для какого-то одного типа транзакций, а потом написать цикл по всем 3 типам.

Дальше предсказания всех 3 моделей нужно сложить, как мы обсуждали выше. Для каждой даты предсказанием будет сумма 3 предсказаний по типам транзакций.

Для итогового прогноза посчитайте MSE. При подсчёте метрики используйте тот же "суммарный" таргет y, что и до этого. Несмотря на то, что мы используем другой подход к решению, сама задача осталась прежней: предсказание суммарной выручки.

Округлите значение до миллионов и введите полученное значение в поле ниже.

In [55]:
query = """
    SELECT payment_date, 
        type,
        -SUM(amount) AS volume
    FROM user_transactions
    WHERE type IN ('basic sale', 'fast sale', 'quick sale')
    GROUP BY payment_date, type
"""

new_df = get_data(query)
new_df.shape[0]

1531

In [57]:
new_df['payment_date'] = pd.to_datetime(new_df['payment_date'], format='%Y-%m-%d')

In [58]:
new_df

,payment_date,type,volume
0,2021-04-27,quick sale,1566030
1,2021-12-29,quick sale,1237786
2,2022-07-11,fast sale,2461446
3,2023-01-08,quick sale,1436429
4,2022-11-27,fast sale,1771166
...,...,...,...
1526,2022-11-20,fast sale,1534786
1527,2021-04-26,quick sale,1752833
1528,2021-10-21,fast sale,1880598
1529,2022-09-28,basic sale,2617040


In [60]:
new_df2 = all_dates.to_frame(name='payment_date').merge(new_df, on='payment_date', how='left').fillna(0)
new_df2['volume'] = new_df2['volume'].astype(int)
new_df2.head(10)

,payment_date,type,volume
0,2021-02-02,fast sale,1352338
1,2021-02-02,basic sale,830390
2,2021-02-02,quick sale,1214139
3,2021-02-03,fast sale,1438638
4,2021-02-03,basic sale,791210
5,2021-02-03,quick sale,1274827
6,2021-02-04,fast sale,1229778
7,2021-02-04,basic sale,819105
8,2021-02-04,quick sale,1272215
9,2021-02-05,fast sale,1436396


In [ ]:
# Отбросим столбец с типом, т.к. он больше не нужен, а пропуски в нём могут вызвать проблемы


In [76]:
preds = {}
targets = {}

for type_ in new_df2.type.unique()[:3]:
    data_ = new_df2[new_df2['type'] == type_]
    data = data_.drop('type', axis=1)
    
    data = all_dates.to_frame(name='payment_date').merge(data, on='payment_date', how='left').fillna(0)
    data['volume'] = data['volume'].astype(int)
    data = generate_features_for_series(data)
    
    X_ = data.drop(['payment_date', 'volume'], axis=1)
    Y_ = data['volume']

    model = LinearRegression()

    model.fit(X_, Y_)
    
    targets[type_] = Y_
    preds[type_] = model.predict(X_)

In [79]:
MSE = round(mean_squared_error(sum(targets.values()), sum(preds.values())) / 1_000_000)
MSE

7648099

### Задача 7. Отдельные модели по типам транзакций (2/2)
Сравните полученные значения MSE для сегментированной и несегментированной моделей.

Почему разница в качестве работы моделей такая мизерная?